# ISSM helper

A collection of functions to help run ISSM a little more intuitively. 

In [ ]:
######################
### ISSM functions ###
######################

from DepthAverage import DepthAverage
from InterpFromGridToMesh import InterpFromGridToMesh
from InterpFromMeshToMesh2d import InterpFromMeshToMesh2d
from project3d import project3d
from project2d import project2d
from setflowequation import setflowequation
from solve import solve

#######################
### python packages ###
#######################

import matplotlib.pyplot as plt
import numpy as np
import os, sys
import rasterio
from rasterio.fill import fillnodata
from rasterio.warp import transform
from shapely import LineString, points, distance, Point, Polygon, contains_xy as contains
from shapely.ops import nearest_points, split
from types import SimpleNamespace

## ```solve_quiet```

Avoid all ISSM's default output, which gets annoying if iterating over multiple timesteps. 

In [ ]:
class Null:
    
    def write(self, *_): pass
    def flush(self): pass

null = Null()

def solve_quiet(md, solver_type):
    
    dn = os.open(os.devnull, os.O_WRONLY)
    o1, o2 = os.dup(1), os.dup(2)
    s1, s2 = sys.stdout, sys.stderr
    sys.stdout = null; sys.stderr = null
    os.dup2(dn, 1); os.dup2(dn, 2); os.close(dn)
    try:
        return solve(md, solver_type)
    finally:
        os.dup2(o1, 1); os.dup2(o2, 2)
        os.close(o1); os.close(o2)
        sys.stdout = s1; sys.stderr = s2

## ```initialize_model```

Set up model parameters that are unlikely to change during a simulation. 

In [ ]:
def initialize_model(md, **kwargs):

    approx = kwargs.get('approximation', None)
    waterline = kwargs.get('waterline', None)
    bed = kwargs.get('bed', None)
    
    T = kwargs.get('temperature', 273.15)

    ice_levelset = kwargs.get('ice_levelset', md.mask.ice_levelset)
    ocean_levelset = kwargs.get('ocean_levelset', md.mask.ocean_levelset)
    Γ_x = kwargs.get('dirichlet_x', np.zeros(md.mesh.numberofvertices).astype(bool))
    Γ_y = kwargs.get('dirichlet_y', np.zeros(md.mesh.numberofvertices).astype(bool))
    Γ_in = kwargs.get('dirichlet_H', np.zeros(md.mesh.numberofvertices).astype(bool))
    
    friction_law = kwargs.get('friction_law', lambda x: x)
    p = kwargs.get('p', 1)
    q = kwargs.get('q', 1)
    
    md.miscellaneous.name = kwargs.get('name', 'name')
    requested_outputs = kwargs.get('requested_outputs', [])
    vertical_layers = kwargs.get('vertical_layers', 5)
    extrusion_exponent = kwargs.get('extrusion_exponent', 1)

    ####################
    ## approximation ###
    ####################

    if approx is None:
        if not (md.flowequation.isHO or md.flowequation.isSSA):
            raise RuntimeError('Choose either "HO" or "SSA" as a flow approximation. This can by done by (a) passing "approximation" as an argument, '
            'or (b) setting setflowequation(md, f"{approximation}", "all").')
        else: 
            md = setflowequation(md, 'HO', 'all') if md.flowequation.isHO else setflowequation(md, 'SSA', 'all')
    else:
        md = setflowequation(md, approx, 'all')
        
    ############################
    ### mesh characteristics ###
    ############################

    num_e = md.mesh.numberofelements
    num_v = md.mesh.numberofvertices
    md.miscellaneous.vertical_layers = vertical_layers if md.flowequation.isHO else np.nan
    md.miscellaneous.extrusion_exponent = extrusion_exponent if md.flowequation.isHO else np.nan
    md.miscellaneous.is3d = True if md.flowequation.isHO else False
    md.miscellaneous.is2d = ~md.miscellaneous.is3d

    if np.all(np.isnan(ice_levelset)) and np.all(np.isnan(ocean_levelset)):
        
        raise RuntimeError('Set an ice and/or ocean levelset. This can be done by (a) passing "ice_levelset" or "ocean_levelset" as arguments, '
        'or (b) setting md.mesh.ice_levelset or md.mesh.ocean_levelset independently. The size of either must be equal to md.mesh.numberofvertices. '
        'For ice_levelset, +/0/- = ice/grounding line/no ice, and for ocean_levelset, +/0/- = grounded/grounding line/floating.')
    
    elif np.all(np.isnan(ice_levelset)):
        
        print('Warning: Assuming the entire domain is ice-covered. Correct by passing ice_levelset as an argument, '
        'or by otherwise setting md.mesh.ice_levelset.')
        ice_levelset = -np.ones_like(ocean_levelset)

    elif np.all(np.isnan(ocean_levelset)):

        if md.flowequation.isSSA:
            print('Warning: Assuming all ice is floating. Correct by passing ocean_levelset as an argument, '
            'or by otherwise setting md.mesh.ocean_levelset.')
            ocean_levelset = -np.ones_like(ice_levelset)
        else:
            print('Warning: Assuming all ice is grounded. Correct by passing ocean_levelset as an argument, '
            'or by otherwise setting md.mesh.ocean_levelset.')
            ocean_levelset = np.ones_like(ice_levelset)

    md.mask.ice_levelset = ice_levelset
    md.mask.ocean_levelset = ocean_levelset 

    ################
    ### geometry ###
    ################

    if waterline is None:
        if md.flowequation.isSSA:
            print('Warning: Assuming waterline is at z = 0.')
            waterline = 0
        else:
            print('Warning: Assuming waterline is below the lowest bed elevation.')
            waterline = np.min(bed) - 1e5
    md.miscellaneous.waterline = waterline

    if bed is None and np.any(md.mask.ocean_levelset > 0):
        raise RuntimeError('Either set a bed elevation or ensure ice is fully floating (with md.mesh.ocean_levelset < 0 everywhere).')
    elif bed is None:
        bed = -1e5
        
    bed = bed*np.ones(num_v) if np.isscalar(bed) else bed
    md.geometry.bed, md.miscellaneous.bed = bed, bed #miscellaneous.bed will store a 2D version even if the mesh becomes extruded

    md.geometry.thickness = np.ones(num_v) #initialize as ones and zeros to first normalize ζ
    md.geometry.surface = np.ones(num_v) #this only matters if the mesh becomes extruded
    md.geometry.base = np.zeros(num_v) #but will be very useful in that case 

    ################
    ### rheology ###
    ################
    
    T = np.array([T]) if np.isscalar(T) else T
    T_t = 263.15 #transition temperature (K)
    Q = 115e3*np.ones_like(T)
    Q[T < T_t] = 6e4
    A_0 = 3.5e-25 #rate prefactor (s^-1 Pa^-3)
    R = 8.314 #gas constant (J mol^-1 K^-1)
    A = A_0*np.exp(-Q/R*(1/T - 1/T_t)) #final rate factor (s^-1 Pa^-3)
    B = A**(-1/md.materials.rheology_n)
    md.materials.rheology_B = B
    md.miscellaneous.rheology_A = A
    md.miscellaneous.temperature = T

    ################
    ### friction ###
    ################

    md.friction.p = p*np.ones(num_e) if np.isscalar(p) else p
    md.friction.q = q*np.ones(num_e) if np.isscalar(q) else q
    md.miscellaneous.friction_p = p
    md.miscellaneous.friction_q = q
    md.miscellaneous.friction_law = friction_law

    ################################
    ### set dirichlet boundaries ###
    ################################

    md.stressbalance.spcvx = np.nan*np.ones(num_v)
    md.stressbalance.spcvy = np.nan*np.ones(num_v)
    md.stressbalance.spcvz = np.nan*np.ones(num_v)
    md.masstransport.spcthickness = np.nan*np.ones(num_v)

    Γ_x = np.isfinite(np.array(Γ_x)) if np.nan in Γ_x else np.array(Γ_x).astype(bool)
    Γ_y = np.isfinite(np.array(Γ_y)) if np.nan in Γ_y else np.array(Γ_y).astype(bool)
    Γ_in = np.isfinite(np.array(Γ_in)) if np.nan in Γ_in else np.array(Γ_in).astype(bool)

    try: 
        md.stressbalance.spcvx[Γ_x] = 1
        md.stressbalance.spcvy[Γ_y] = 1
        md.masstransport.spcthickness[Γ_in] = 1
    except:
        proj = lambda x: project3d(md, 'vector', x, 'type', 'node')
        md.stressbalance.spcvx[proj(Γ_x)] = 1
        md.stressbalance.spcvy[proj(Γ_y)] = 1
        md.masstransport.spcthickness[proj(Γ_in)] = 1     
        
    ##############
    ### extras ###
    ##############

    requested_outputs = list(requested_outputs)
    requested_outputs += ['default', 'StrainRatexx', 'StrainRateyy', 'StrainRatexy']
    if md.flowequation.isHO:
        requested_outputs += ['StrainRatexz', 'StrainRateyz', 'StrainRatezz']
    md.stressbalance.requested_outputs = requested_outputs
    md.miscellaneous.requested_outputs = requested_outputs

    md.stressbalance.referential = np.nan*np.ones((num_v, 6))
    md.stressbalance.loadingforce = np.zeros((num_v, 3))
    md.basalforcings.groundedice_melting_rate = np.zeros(num_v)
    md.basalforcings.floatingice_melting_rate = np.zeros(num_v)
    md.smb.initialize(md) 

    ############################
    ### extrude if necessary ###
    ############################

    if not hasattr(md.mesh, 'x2d'):
        md.miscellaneous.mesh2d = md.mesh
        if md.flowequation.isHO:
            md = md.extrude(vertical_layers, extrusion_exponent, 1)
            md.miscellaneous.zeta = md.mesh.z #this will tell us how z is distributed proportionally
            md.miscellaneous.is3d = True
            md.miscellaneous.is2d = False

## ```diagnostic_solve```

Use mirrors ```icepack```'s function of the same name. 

In [ ]:
def diagnostic_solve(md, **kwargs):
    
    H = kwargs['thickness']
    bed = kwargs.get('bed', md.geometry.bed)
    waterline = kwargs.get('waterline', md.miscellaneous.waterline)
    u = kwargs.get('velocity', None)
    C = kwargs.get('friction', None)
    quiet = kwargs.get('quiet', True)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    proj2 = lambda x: project2d(md, x, 1)
    proj3 = lambda x: project3d(md, 'vector', x, 'type', 'node')
    ice_mask = md.mask.ice_levelset <= 0
    ice_mask = proj2(ice_mask) if md.miscellaneous.is3d else ice_mask

    ####################
    ### set geometry ###
    ####################

    H[(~ice_mask) | (H <= 0)] = 1e-3
    if md.miscellaneous.is3d:
        bed = proj2(bed)
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]

    if md.miscellaneous.is3d:
        H, b = proj3(H), proj3(b)

    md.geometry.thickness = H
    md.geometry.base = b
    md.geometry.surface = b + H

    ######################
    ### basal friction ###
    ######################

    if C is None:
        if md.flowequation.isSSA and np.all(md.mask.ocean_levelset <= 0):
            C = 0
        else:
            raise RuntimeError('Must specify friction coefficient.')
    
    friction_law = md.miscellaneous.friction_law
    C = friction_law(C)
    md.friction.coefficient = C*np.ones(md.mesh.numberofvertices) if np.isscalar(C) else C
    md.friction.coefficient[float_mask] = 0.

    ####################################
    ### velocity boundary conditions ###
    ####################################

    Γ_x = np.isfinite(md.stressbalance.spcvx)
    Γ_y = np.isfinite(md.stressbalance.spcvy)

    if True in Γ_x | Γ_y:
        if u is None:
            raise RuntimeError('Must provide initial velocity to enforce Dirichlet BCs.')
        else: 
            md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
            md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]

    #####################
    ### update mesh.z ###
    #####################

    if md.miscellaneous.is3d:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Stressbalance') if quiet else solve(md, 'Stressbalance')
    solution = md.results.StressbalanceSolution

    u_x, u_y = solution.Vx.flatten(), solution.Vy.flatten()
    u = (u_x, u_y)
    if md.miscellaneous.is3d:
        u = (u_x, u_y, solution.Vz.flatten())

    ######################
    ### postprocessing ###
    ######################

    outputs = md.stressbalance.requested_outputs
    requested = [o for o in outputs if o != 'default']
    md.miscellaneous.outputs = SimpleNamespace(**{name: getattr(solution, name) for name in requested})
        
    ##############
    ### return ###
    ##############
    
    return u          

## ```prognostic_solve```

Use mirrors ```icepack```'s function of the same name, but returns both thickness and surface elevation fields. 

In [ ]:
def prognostic_solve(md, Δt, **kwargs):
    
    H = kwargs['thickness']
    u = kwargs['velocity']
    
    bed = kwargs.get('bed', md.geometry.bed)
    waterline = kwargs.get('waterline', md.miscellaneous.waterline)
    H_in = kwargs.get('thickness_inflow', None)
    quiet = kwargs.get('quiet', True)
    stab = kwargs.get('stabilization', 0)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    proj2 = lambda x: project2d(md, x, 1)
    proj3 = lambda x: project3d(md, 'vector', x, 'type', 'node')

    ###########################
    ### resize if necessary ###
    ###########################

    items = [H, H_in]
    if md.miscellaneous.is3d:
        for i, item in enumerate(items):
            if (not np.isscalar(item)) and (np.size(item) < md.mesh.numberofvertices):
                items[i] = proj3(item)
    H, H_in = items    

    ####################
    ### set geometry ###
    ####################

    ice_mask = md.mask.ice_levelset <= 0
    H[(~ice_mask) | (H <= 0)] = 1e-3
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]

    #####################
    ### update mesh.z ###
    #####################

    if md.miscellaneous.is3d:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ##################################
    ### inflow boundary conditions ###
    ##################################

    Γ_in = np.isfinite(md.masstransport.spcthickness)
    
    if True in Γ_in:
        if H_in is None:
            raise RuntimeError('Must provide thickness constraint to enforce Dirichlet BC.')
        else:
            md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]

    ############################
    ### transient patameters ###
    ############################

    md.timestepping.time_step = Δt #try to model only a single mass-transport step
    md.timestepping.final_time = Δt
    md.transient.isstressbalance = 0 #solve the stress balance?
    md.transient.ismasstransport = 1 #solve mass transport?
    md.transient.isthermal = 0 #don't bother with heat transport
    md.initialization.vx = u[0]
    md.initialization.vy = u[1]
    if stab is not None:
        md.masstransport.stabilization = stab

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    H = solution.Thickness.flatten()

    ################################
    ### update floating levelset ###
    ################################

    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]
    s = b + H

    md.geometry.thickness = H
    md.geometry.base = b
    md.geometry.surface = s
    
    ocean_mask = (~ice_mask) | float_mask
    md.mask.ocean_levelset = np.where(ocean_mask, -1.0, 1.0)

    ##############
    ### output ###
    ##############

    if md.miscellaneous.is3d:
        H, s = proj2(H), proj2(s)
    return H, s

## ```coupled_solve```

Performs both diagnostic and prognostic solves in a single step. Somewhat faster that splitting. Output is the pair ```(u, H, s)```, where the dimension of ```u``` depends on the approximation.  

In [ ]:
def coupled_solve(md, Δt, **kwargs):
    
    H = kwargs['thickness']
    u = kwargs['velocity']
    
    bed = kwargs.get('bed', md.geometry.bed)
    waterline = kwargs.get('waterline', md.miscellaneous.waterline)
    H_in = kwargs.get('thickness_inflow', None)
    quiet = kwargs.get('quiet', True)
    stab = kwargs.get('stabilization', 0)
    C = kwargs.get('friction', None)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    proj2 = lambda x: project2d(md, x, 1)
    proj3 = lambda x: project3d(md, 'vector', x, 'type', 'node')

    ###########################
    ### resize if necessary ###
    ###########################

    items = [H, H_in]
    if md.miscellaneous.is3d:
        for i, item in enumerate(items):
            if (not np.isscalar(item)) and (np.size(item) < md.mesh.numberofvertices):
                items[i] = proj3(item)
    H, H_in = items    

    ####################
    ### set geometry ###
    ####################

    ice_mask = md.mask.ice_levelset <= 0
    H[(~ice_mask) | (H <= 0)] = 1e-3
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]
    s = b + H

    md.geometry.thickness = H
    md.geometry.base = b
    md.geometry.surface = s


    ######################
    ### basal friction ###
    ######################

    if C is None:
        if md.flowequation.isSSA:
            C = 0
        else:
            raise RuntimeError('Must specify friction coefficient.')
    
    friction_law = md.miscellaneous.friction_law
    C = friction_law(C)
    md.friction.coefficient = C*np.ones(md.mesh.numberofvertices) if np.isscalar(C) else C
    md.friction.coefficient[float_mask] = 0.

    #####################
    ### update mesh.z ###
    #####################

    if md.miscellaneous.is3d:
        ζ = md.miscellaneous.zeta
        md.mesh.z = b + ζ*H

    ###########################
    ### boundary conditions ###
    ###########################

    Γ_x = np.isfinite(md.stressbalance.spcvx)
    Γ_y = np.isfinite(md.stressbalance.spcvy)
    Γ_in = np.isfinite(md.masstransport.spcthickness)

    if True in Γ_x | Γ_y:
        if u is None:
            raise RuntimeError('Must provide initial velocity to enforce Dirichlet BCs.')
        else: 
            md.stressbalance.spcvx[Γ_x] = u[0] if np.isscalar(u[0]) else u[0][Γ_x]
            md.stressbalance.spcvy[Γ_y] = u[1] if np.isscalar(u[1]) else u[1][Γ_y]

    if True in Γ_in:
        if H_in is None:
            raise RuntimeError('Must provide thickness constraint to enforce Dirichlet BC.')
        else:
            md.masstransport.spcthickness[Γ_in] = H_in if np.isscalar(H_in) else H_in[Γ_in]

    ############################
    ### transient patameters ###
    ############################

    md.timestepping.time_step = Δt #try to model only a single mass-transport step
    md.timestepping.final_time = Δt
    md.transient.isstressbalance = 1 #solve the stress balance?
    md.transient.ismasstransport = 1 #solve mass transport?
    md.transient.isthermal = 0 #don't bother with heat transport
    md.initialization.vx = u[0]
    md.initialization.vy = u[1]
    if stab is not None:
        md.masstransport.stabilization = stab

    ##################
    ### solve step ###
    ##################

    md = solve_quiet(md, 'Transient') if quiet else solve(md, 'Transient')
    solution = md.results.TransientSolution[-1]
    
    u_x, u_y = solution.Vx.flatten(), solution.Vy.flatten()
    u = (u_x, u_y)
    if md.miscellaneous.is3d:
        u = (u_x, u_y, solution.Vz.flatten())
    H = solution.Thickness.flatten()

    ######################
    ### postprocessing ###
    ######################

    outputs = md.stressbalance.requested_outputs
    requested = [o for o in outputs if o != 'default']
    md.miscellaneous.outputs = SimpleNamespace(**{name: getattr(solution, name) for name in requested})

    ################################
    ### update floating levelset ###
    ################################

    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]
    s = b + H

    md.geometry.thickness = H
    md.geometry.base = b
    md.geometry.surface = s
    
    ocean_mask = (~ice_mask) | float_mask
    md.mask.ocean_levelset = np.where(ocean_mask, -1.0, 1.0)

    ##############
    ### output ###
    ##############
    
    if md.miscellaneous.is3d:
        H, s = proj2(H), proj2(s)
    return u, H, s

## ```order_boundary```

Sometimes it is necessary to order boundary nodes by looping from one end of a terminus to the other along the glacier boundary. For example, may want to specify a contiguous section as a named boundary for imposing Dirichlet BCs, or may need to update an ice mask after moving a terminus. 

In [ ]:
def order_boundary(md, **kwargs):
    
    mesh2d = md.miscellaneous.mesh2d if hasattr(md.miscellaneous, 'mesh2d') else md.mesh
    x, y = mesh2d.x, mesh2d.y
    bnd = mesh2d.segments[:, :2] - 1
    buf = kwargs.get('buffer', 1.0)

    adj = {}
    for i, j in bnd:
        adj.setdefault(i, []).append(j)
        adj.setdefault(j, []).append(i)

    nodes = np.fromiter(adj.keys(), int)

    # pick a deterministic start: leftmost, then lowest
    start = nodes[np.lexsort((y[nodes], x[nodes]))][0]

    path = [start]
    prev, cur = -1, start
    while True:
        nxts = adj[cur]
        nxt = nxts[0] if nxts[0] != prev else nxts[1]
        if nxt == start:
            break
        path.append(nxt)
        prev, cur = cur, nxt

    bnd_nodes = np.asarray(path)
    md.miscellaneous.bnd_nodes = bnd_nodes
    Ω = Polygon(np.column_stack((x[bnd_nodes], y[bnd_nodes]))).buffer(buf)
    md.miscellaneous.domain = Ω
    return bnd_nodes

## ```plot_boundary```

Plot a subset of (already-ordered) boundary from note n to note m. This is necessary for manually chooing the appropriate segments for a given boundary condition. 

In [ ]:
def plot_boundary(md, **kwargs):
    
    index_1 = kwargs.get('i', 0)
    index_2 = kwargs.get('j', None)

    figsize = kwargs.get('figsize', (8, 8))
    linewidth = kwargs.get('linewidth', 0.2)
    color = kwargs.get('color', 'k')
    lw = kwargs.get('lw', 2)
    num_ticks = kwargs.get('num_ticks', 50)
    fontsize = kwargs.get('fontsize', 12)

    mesh2d = md.miscellaneous.mesh2d if hasattr(md.miscellaneous, 'mesh2d') else md.mesh
    x, y, tri = mesh2d.x, mesh2d.y, mesh2d.elements - 1

    bnd_nodes = order_boundary(md)
    subset = bnd_nodes[index_1:index_2]

    fig, ax = plt.subplots(figsize = figsize)
    ax.triplot(x, y, tri, linewidth = linewidth)
    ax.plot(x[subset], y[subset], lw = lw, color = color)

    for k in range(0, len(bnd_nodes), num_ticks):
        ax.text(x[bnd_nodes[k]], y[bnd_nodes[k]], str(k), fontsize = fontsize)

    ax.set_aspect('equal')
    plt.show()

## ```extend_terminus```

To ensure that the terminus splits the meshed domain, it may at times be necessary to extend the line segment slightly. 

In [ ]:
def extend_terminus(md, **kwargs):

    terminus = kwargs['terminus']
    ε = kwargs.get('ε', kwargs.get('epsilon', 10.0))

    Ω = getattr(md.miscellaneous, 'domain', None)
    if Ω is None:
        order_boundary(md)
        Ω = md.miscellaneous.domain

    coords = list(terminus.coords)

    p0 = Point(coords[0])
    if Ω.covers(p0):
        q0 = nearest_points(p0, Ω.boundary)[1]
        p0 = np.asarray(p0.coords[0])
        q0 = np.asarray(q0.coords[0])
        d0 = (q0 - p0)/np.linalg.norm(q0 - p0)
        q0 = q0 + ε*d0
        coords[0:0] = [(q0[0], q0[1])]

    pn = Point(coords[-1])
    if Ω.covers(pn):
        qn = nearest_points(pn, Ω.boundary)[1]
        pn = np.asarray(pn.coords[0])
        qn = np.asarray(qn.coords[0])
        dn = (qn - pn)/np.linalg.norm(qn - pn)
        qn = qn + ε*dn
        coords.append((qn[0], qn[1]))

    return LineString(coords)

## ```advance_terminus```

Moves the terminus forward in accordance with the (depth-averaged) frontal velocity field. 

In [ ]:
def advance_terminus(md, Δt, **kwargs):

    H = kwargs['thickness']
    u = kwargs['velocity']
    terminus = kwargs['terminus']

    bed = kwargs.get('bed', md.geometry.bed)
    waterline = kwargs.get('waterline', md.miscellaneous.waterline)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    proj2 = lambda x: project2d(md, x, 1)
    proj3 = lambda x: project3d(md, 'vector', x, 'type', 'node')

    ###########################
    ### resize if necessary ###
    ###########################

    ice_mask = md.mask.ice_levelset <= 0
    items = [H, bed, ice_mask]
    
    if md.miscellaneous.is3d:
        for i, item in enumerate(items):
            if np.size(item) == md.mesh.numberofvertices:
                items[i] = proj2(item)
    
    H, bed, ice_mask = items
    old_ice = np.copy(ice_mask)
    
    ####################
    ### set geometry ###
    ####################

    H[H <= 0] = 1e-3
    H[(~ice_mask) | (H <= 0)] = 1e-3
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]

    #######################
    ### advect terminus ###
    #######################
    
    u_x = DepthAverage(md, u[0]) if md.miscellaneous.is3d else u[0]
    u_y = DepthAverage(md, u[1]) if md.miscellaneous.is3d else u[1]
    x, y = md.miscellaneous.mesh2d.x, md.miscellaneous.mesh2d.y
    elts = md.miscellaneous.mesh2d.elements
    xs, ys = map(np.asarray, terminus.xy)
    
    x_shift = InterpFromMeshToMesh2d(elts, x, y, u_x*Δt, xs, ys).ravel()
    y_shift = InterpFromMeshToMesh2d(elts, x, y, u_y*Δt, xs, ys).ravel()

    terminus_curve = LineString(np.column_stack((xs + x_shift, ys + y_shift)))

    c = np.column_stack(terminus_curve.xy)
    c[1:-1] = 0.5*c[1:-1] + 0.25*(c[:-2] + c[2:])   # 1 smoothing pass, endpoints fixed
    terminus_curve = LineString(c)


    ############################
    ### reparameterize curve ###
    ############################
    
    N = len(xs)
    L = terminus_curve.length
    ts = np.linspace(0.0, L, N)
    terminus_curve = LineString([terminus_curve.interpolate(t) for t in ts])

    ######################
    ### domain polygon ###
    ######################
    
    Ω = getattr(md.miscellaneous, 'domain', None)
    if Ω is None:
        order_boundary(md)
        Ω = md.miscellaneous.domain

    parts = list(split(Ω, terminus_curve).geoms)
    if len(parts) < 2:
        terminus_curve = extend_terminus(md, terminus = terminus_curve)
        parts = list(split(Ω, terminus_curve).geoms)
        if len(parts) < 2:
            raise RuntimeError('Split failed.')

    p0 = Point(x[old_ice][0], y[old_ice][0])
    Ω_ice = parts[0] if parts[0].covers(p0) else parts[1]

    ###########################
    ### update ice levelset ###
    ###########################
    
    dist = np.array([terminus_curve.distance(Point(xi, yi)) for xi, yi in zip(x, y)])
    inside = np.array([Ω_ice.covers(Point(xi, yi)) for xi, yi in zip(x, y)])
    ice_levelset = np.where(inside, -dist, dist)
    md.mask.ice_levelset = proj3(ice_levelset) if md.miscellaneous.is3d else ice_levelset
    new_ice = inside & (~old_ice)

    ########################
    ### thickness infill ###
    ########################
    
    H_term = InterpFromMeshToMesh2d(elts, x, y, H, xs, ys).ravel()
    
    if np.any(new_ice):
        ds = np.sqrt(np.diff(xs)**2 + np.diff(ys)**2)
        s_nodes = np.r_[0.0, np.cumsum(ds)]
        term_old = LineString(np.column_stack((xs, ys)))
    
        s_proj = np.array([term_old.project(Point(xi, yi)) for xi, yi in zip(x[new_ice], y[new_ice])])
        H[new_ice] = np.interp(s_proj, s_nodes, H_term)

    ########################################
    ### isostacy and levelset adjustment ###
    ########################################
        
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]
    s = b + H

    ice_mask = ice_levelset <= 0
    ocean_mask = (~ice_mask) | float_mask
    ocean_levelset = np.where(ocean_mask, -1.0, 1.0)
    md.mask.ocean_levelset = proj3(ocean_levelset) if md.miscellaneous.is3d else ocean_levelset

    ##############
    ### output ###
    ##############
    
    md.geometry.thickness = H
    md.geometry.surface = s
    md.geometry.base = b

    return terminus_curve, H.flatten(), s.flatten()

## ```simple_calve```

Calves a semicircle. 

In [ ]:
def simple_calve(md, **kwargs):

    r = kwargs['radius']
    terminus = kwargs['terminus']
    H = kwargs['thickness']

    bed = kwargs.get('bed', md.geometry.bed)
    waterline = kwargs.get('waterline', md.miscellaneous.waterline)

    ρ_i = md.materials.rho_ice
    ρ_w = md.materials.rho_water
    ϱ = ρ_i/ρ_w

    proj2 = lambda x: project2d(md, x, 1)
    proj3 = lambda x: project3d(md, 'vector', x, 'type', 'node')

    x, y = md.miscellaneous.mesh2d.x, md.miscellaneous.mesh2d.y
    xs, ys = map(np.asarray, terminus.xy)

    ###########################
    ### resize if necessary ###
    ###########################

    ice_mask = md.mask.ice_levelset <= 0
    items = [H, bed, ice_mask]
    
    if md.miscellaneous.is3d:
        for i, item in enumerate(items):
            if np.size(item) == md.mesh.numberofvertices:
                items[i] = proj2(item)
    
    H, bed, ice_mask = items
    old_ice = np.copy(ice_mask)

    ####################
    ### set geometry ###
    ####################

    H[H <= 0] = 1e-3
    H[(~ice_mask) | (H <= 0)] = 1e-3
    float_mask = (bed - waterline) < (-ϱ*H)
    b = bed.copy()
    b[float_mask] = waterline - ϱ*H[float_mask]
    s = b + H

    ######################
    ### domain polygon ###
    ######################

    Ω = getattr(md.miscellaneous, 'domain', None)
    if Ω is None:
        order_boundary(md)
        Ω = md.miscellaneous.domain

    parts = list(split(Ω, terminus).geoms)
    if len(parts) < 2:
        terminus = extend_terminus(md, terminus = terminus)
        parts = list(split(Ω, terminus).geoms)
        if len(parts) < 2:
            raise RuntimeError('Split failed.')

    p0 = Point(x[old_ice][0], y[old_ice][0])
    Ω_ice_old = parts[0] if parts[0].covers(p0) else parts[1]

    ##############################
    ### carve semicircle in curve
    ##############################

    c = np.column_stack((xs, ys))
    mid = c[len(c)//2]
    d = c - mid

    t = c[-1] - c[0]
    t = t/np.linalg.norm(t)
    n = np.array([-t[1], t[0]])

    eps = 1e-6*r + 1.0
    n_in = n if Ω_ice_old.covers(Point(*(mid + eps*n))) else -n

    dt = d @ t
    mask = np.abs(dt) <= r
    indent = np.sqrt(np.maximum(0.0, r*r - dt[mask]**2))

    c[mask] = c[mask] + indent[:, None]*n_in[None, :]
    c[0] = np.r_[xs[0], ys[0]]
    c[-1] = np.r_[xs[-1], ys[-1]]

    terminus = LineString(c)

    ############################
    ### smoothing + resample ###
    ############################

    c = np.column_stack(terminus.xy)
    c[1:-1] = 0.5*c[1:-1] + 0.25*(c[:-2] + c[2:])
    terminus = LineString(c)

    N = len(xs)
    L = terminus.length
    ts = np.linspace(0.0, L, N)
    terminus = LineString([terminus.interpolate(t) for t in ts])

    #######################
    ### updated polygon ###
    #######################

    parts = list(split(Ω, terminus).geoms)
    if len(parts) < 2:
        terminus = extend_terminus(md, terminus = terminus)
        parts = list(split(Ω, terminus).geoms)
        if len(parts) < 2:
            raise RuntimeError('Split failed.')

    Ω_ice = parts[0] if parts[0].covers(p0) else parts[1]

    ###########################
    ### update ice levelset ###
    ###########################

    dist = np.array([terminus.distance(Point(xi, yi)) for xi, yi in zip(x, y)])
    inside = np.array([Ω_ice.covers(Point(xi, yi)) for xi, yi in zip(x, y)])
    ice_levelset = np.where(inside, -dist, dist)
    md.mask.ice_levelset = proj3(ice_levelset) if md.miscellaneous.is3d else ice_levelset

    ###############################
    ### calved-off: thin + flat ###
    ###############################

    calved = old_ice & (~inside)
    H[calved] = 1e-3
    s[calved] = waterline
    b = s - H

    md.geometry.thickness = proj3(H) if md.miscellaneous.is3d else H
    md.geometry.surface = proj3(s) if md.miscellaneous.is3d else s
    md.geometry.base = proj3(b) if md.miscellaneous.is3d else b

    return terminus, H.flatten(), s.flatten()

## ```interpolate_raster```

Raster to mesh gridpoints. 

In [ ]:
def interpolate_raster(md, raster, **kwargs):

    mesh = md.miscellaneous.mesh2d if hasattr(md.miscellaneous, 'mesh2d') else md.mesh
    x, y = mesh.x, mesh.y

    fill = kwargs.get('fill', True)
    epsg = kwargs.get('epsg', 32645)
    mesh_crs = f'EPSG:{epsg}'   

    with rasterio.open(raster) as src:

        z = src.read(1).astype(float)
        t = src.transform
        ny, nx = z.shape
        src_crs = src.crs

        if fill:
            z = fillnodata(z, mask = np.isfinite(z))

        cols = np.arange(nx) + 0.5
        rows = np.arange(ny) + 0.5

        xs = t.c + cols*t.a
        ys = t.f + rows*t.e

        if ys[1] < ys[0]:
            ys = ys[::-1]
            z = z[::-1, :]

        # Transform mesh → raster CRS if needed
        if src_crs is not None and str(src_crs) != mesh_crs:
            x, y = transform(mesh_crs, src_crs, x.tolist(), y.tolist())
            x = np.asarray(x)
            y = np.asarray(y)

    vals = InterpFromGridToMesh(xs, ys, z, x, y, np.nan)
    return vals

## ```effective_strain_rate```

In [ ]:
def effective_strain_rate(md, **kwargs):

    depth_averaged = kwargs.get('depth_averaged', False) if md.flowequation.isHO else False #SSA is already dpth-averaged by default
    out = md.miscellaneous.outputs
    ε_xx, ε_xy, ε_yy = out.StrainRatexx, out.StrainRatexy, out.StrainRateyy
    
    inner_ε =  ε_xx**2 + 2*ε_xy**2 + ε_yy**2
    trace_ε = ε_xx + ε_yy
    
    if md.flowequation.isHO:
        ε_xz, ε_yz, ε_zz = out.StrainRatexz, out.StrainRateyz, out.StrainRatezz
        inner_ε += ε_zz**2 + 2*ε_xz**2 + 2*ε_yz**2
        trace_ε += ε_zz
    
    ε_e = np.sqrt(1/2*(inner_ε + trace_ε**2)) #generally true since the trace term should be zero in 3D
    ε_e = ε_e if not depth_averaged else DepthAverage(md, ε_e)

    return ε_e.flatten()    

## Convert to .py script

In [ ]:
# !jupyter nbconvert --to script issm_helper.ipynb